In [1]:
from functools import partial
import json
import math
import re

import numexpr
import numpy as np
import pandas as pd

### Known issues
* Raiseskeleton life is high compared to game, but matches pet calculator
* Claygolem AR is 20 lower than game. Much lower than pet calculator, which uses character level.

In [2]:
N_CALC_FIELDS = 4
N_PARAMS = 8
LVL_BREAKPOINTS = [(0, 1), (1, 8), (8, 16), (16, 22), (22, 28), (28, None)]

# Read game data JSON
Extracted from patch_d2.mpq and processed by mpq_data_parser

* Corrected Firewall EDmgSymPerCalc, added )
* Adjusted Raise Skeleton descline, removed outer (), was breaking calc regexes
* Adjusted Zeal descline, removed outer (), was breaking calc regexes

In [3]:
with open('../mpq_data_parser/data/d2_skill_data.json') as f:
    skill_details = json.load(f)['skillDetails']

# Generate descriptions from lookup

## Define calculators

In [4]:
def calculate_damage(skill, skill_levels, lvl, initial_damage_key, damage_per_level_key_root, synergy_key, missile_num=None):
    if missile_num:
        skill = skill[f'missile{missile_num}']
    
    synergy_bonus = calculate_skill_value(skill.get(synergy_key), skill, skill_levels, lvl) or 0
    synergy_multiplier = (100 + synergy_bonus) / 100
    
    damage_per_level_values = (
        [skill[initial_damage_key]] + [skill.get(f'{damage_per_level_key_root}{i}', 0) for i in range(1, 6)]
    )
    damage = 0
    for (lower, upper), damage_per_level in zip(LVL_BREAKPOINTS, damage_per_level_values):
        if lvl <= lower:
            break
        lvl_for_band = min(upper, lvl) if upper is not None else lvl
        damage += (lvl_for_band - lower) * damage_per_level
    hitshift = skill.get('hitShift', 0)
    return damage * synergy_multiplier * 2 ** (hitshift - 8)

calculate_physical_damage_min = partial(
    calculate_damage,
    initial_damage_key='minDam',
    damage_per_level_key_root='minLevDam',
    synergy_key='dmgSymPerCalc',
)
calculate_physical_damage_max = partial(
    calculate_damage,
    initial_damage_key='maxDam',
    damage_per_level_key_root='maxLevDam',
    synergy_key='dmgSymPerCalc',
)
calculate_elemental_damage_min = partial(
    calculate_damage,
    initial_damage_key='eMin',
    damage_per_level_key_root='eMinLev',
    synergy_key='eDmgSymPerCalc',
)
calculate_elemental_damage_max = partial(
    calculate_damage,
    initial_damage_key='eMax',
    damage_per_level_key_root='eMaxLev',
    synergy_key='eDmgSymPerCalc',
)

In [5]:
def calculate_mana_cost_per_second(skill, skill_levels, lvl):
    return calculate_mana_cost(skill, skill_levels, lvl) * FRAMES_PER_SECOND / 2

def calculate_mana_cost(skill, skill_levels, lvl, multiplier=None):
    mana, lvl_mana, mana_shift = skill['mana'], skill.get('lvlMana', 0), skill['manaShift']
    cost = (mana + lvl_mana * (lvl - 1)) * 2 ** (mana_shift - 8)
    if multiplier:
        cost *= multiplier
    return max(cost, skill.get('minMana', 0))

def floor(x, precision=None):
    if precision is None:
        return math.floor(x)
    floored = math.floor(precision * x) / precision
    return int_if_no_decimals(floored)

def int_if_no_decimals(x):
    return int(x) if not x % 1 else x

def calculate_to_hit(skill, skill_levels, lvl):
    to_hit_expression = skill.get('toHitCalc')
    if to_hit_expression:
        return calculate_skill_value(to_hit_expression, skill, skill_levels, lvl)

    return skill.get('toHit', 0) + skill.get('levToHit', 0) * (lvl - 1)

def calculate_length(skill, skill_levels, lvl):
    synergy_length_bonus = calculate_skill_value(skill.get('eLenSymPerCalc'), skill, skill_levels, lvl) or 0
    synergy_length_multiplier = (100 + synergy_length_bonus) / 100

    length_per_level_values = (
        [skill['eLen']] + [skill.get(f'eLevLen{i}', 0) for i in range(1, 4)]
    )
    length = 0
    for (lower, upper), length_per_level in zip(LVL_BREAKPOINTS[:3], length_per_level_values):
        if lvl <= lower:
            break
        lvl_for_band = min(upper, lvl) if upper is not None else lvl
        length += (lvl_for_band - lower) * length_per_level

    return length * synergy_length_multiplier

def calculate_mXeo(skill, skill_levels, lvl, missile_num):
    return 256 * calculate_elemental_damage_min(skill, skill_levels, lvl, missile_num=missile_num)

def calculate_mXey(skill, skill_levels, lvl, missile_num):
    return 256 * calculate_elemental_damage_max(skill, skill_levels, lvl, missile_num=missile_num)

def create_linear_calculator(param_key_a, param_key_b):
    def calculator(skill, skill_levels, lvl):
        a = skill['params'].get(param_key_a, 0)
        b = skill['params'].get(param_key_b, 0)
        return a + b * (lvl - 1)
    return calculator

def create_diminishing_calculator(param_key_a, param_key_b):
    def calculator(skill, skill_levels, lvl):
        a = skill['params'].get(param_key_a, 0)
        b = skill['params'].get(param_key_b, 0)
        return floor(a + ((110 * lvl) * (b - a)) / (100 * (lvl + 6)))
    return calculator

def create_calculator_on_calc_field(calc_field):
    def calculator(skill, skill_levels, lvl):
        return calculate_skill_value(skill['calcs'][calc_field], skill, skill_levels, lvl)
    return calculator

def create_mastery_calculator(mastery_key):
    def calculator(skill, skill_levels, lvl):
        return calculate_skill_value(skill['mastery'][mastery_key], skill, skill_levels, lvl)
    return calculator

def _get_param(skill, skill_levels, lvl, param_number):
    return skill['params'][f'par{param_number}']

In [6]:
def calculate_skill_value(calc_expression, skill, skill_levels, lvl):
    if calc_expression is None:
        return None
    if isinstance(calc_expression, int):
        return calc_expression
    if not calc_expression.strip():
        return None
    
    calc_expression = _fill_other_skill_levels(str(calc_expression), skill_levels)
    calc_expression = _evaluate_sklvl_calcs(calc_expression, skill, skill_levels, lvl)
    calc_expression = _evaluate_other_entity_calcs(calc_expression, skill, skill_levels, lvl)
    calc_expression = _evaluate_calcs(calc_expression, skill, skill_levels, lvl)
    calc_expression = _evaluate_mins(calc_expression)
    calc_expression = _fill_if_thens(calc_expression)

    calc = numexpr.evaluate(calc_expression).item()
    return calc

def _fill_other_skill_levels(calc_expression, skill_levels):
    return re.sub(
        pattern=r"skill\('((?:\w|\s)+)'.(?:lvl|blvl)\)",
        repl=partial(_matched_skill_level, skill_levels=skill_levels),
        string=calc_expression,
    )

def _evaluate_sklvl_calcs(calc_expression, skill, skill_levels, lvl):
    return re.sub(
        pattern=r"sklvl\('((?:\w|\s)+)'\.(\w+)\.(?!lvl)(\w+)\)",
        repl=partial(_matched_sklvl_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=calc_expression,
    )

def _evaluate_other_entity_calcs(calc_expression, skill, skill_levels, lvl):
    return re.sub(
        pattern=r"(skill|miss)\('((?:\w|\s)+)'\.(?!lvl)(\w+)\)",
        repl=partial(_matched_other_entity_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=calc_expression,
    )

def  _evaluate_calcs(calc_expression, skill, skill_levels, lvl):
    return re.sub(
        pattern='|'.join(CALC_LOOKUP.keys()),
        repl=partial(_matched_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=calc_expression,
    )

def _fill_if_thens(calc_expression):
    return re.sub(
        pattern=r'\(?(\(.*\))\s\?\s?(.*)\s?:\s?(\(?[^\.|+]*\)?)',
        repl=lambda match: match[2] if numexpr.evaluate(match[1]).item() else match[3],
        string=calc_expression,
    )

def _evaluate_mins(calc_expression):
    RE_MATH_SYMBOLS = '(?:\d|\+|-|\*|\/|\.|\s|\(|\))'
    return re.sub(
        pattern=fr"min\(({RE_MATH_SYMBOLS}+),\s*({RE_MATH_SYMBOLS}+)\)",
        repl=_matched_min_function,
        string=calc_expression,
    )

def _matched_skill_level(match, skill_levels):
    return str(skill_levels.get(match[1], 0))

def _matched_sklvl_calc(match, skill, skill_levels, lvl):
    other_skill = skill[f'relatedSkills'][match[1]]
    lvl_calculator = CALC_LOOKUP[match[2]]
    calculator = CALC_LOOKUP[match[3]]

    effective_lvl = lvl_calculator(skill, skill_levels, lvl)
    return str(calculator(other_skill, skill_levels, effective_lvl))

def _matched_other_entity_calc(match, skill, skill_levels, lvl):
    entity_kind = {'miss': 'Missile', 'skill': 'Skill'}[match[1]]
    entity_name = match[2]
    entity = skill[f'related{entity_kind}s'][entity_name]
    
    calculator = CALC_LOOKUP[match[3]]
    lvl = skill_levels.get(entity_name, 0) if entity_kind == 'Skill' else lvl
    return str(calculator(entity, skill_levels, lvl))
        
def _matched_calc(match, skill, skill_levels, lvl):
    calculator = CALC_LOOKUP[match[0]]
    return str(calculator(skill, skill_levels, lvl))

def _matched_min_function(match):
    return min(match[1], match[2])

In [7]:
CALC_LOOKUP = {
    'toht': calculate_to_hit,
    'edmn': calculate_elemental_damage_min,
    'edmx': calculate_elemental_damage_max,
    'edln': calculate_length,
    'edns': lambda skill, skill_levels, lvl: 256 * calculate_elemental_damage_min(skill, skill_levels, lvl),
    'edxs': lambda skill, skill_levels, lvl: 256 * calculate_elemental_damage_max(skill, skill_levels, lvl),
    'ln12': create_linear_calculator('par1', 'par2'),
    'ln34': create_linear_calculator('par3', 'par4'),
    'ln56': create_linear_calculator('par5', 'par6'),
    'ln78': create_linear_calculator('par7', 'par8'),
    'dm12': create_diminishing_calculator('par1', 'par2'),
    'dm34': create_diminishing_calculator('par3', 'par4'),
    'dm56': create_diminishing_calculator('par5', 'par6'),
    'dm78': create_diminishing_calculator('par7', 'par8'),
    'math': create_mastery_calculator('passiveMasteryTh'),
    'madm': create_mastery_calculator('passiveMasteryDmg'),
    'macr': create_mastery_calculator('passiveMasteryCrit'),
    'm1eo': partial(calculate_mXeo, missile_num=1),
    'm1ey': partial(calculate_mXey, missile_num=1),
    'm2eo': partial(calculate_mXeo, missile_num=2),
    'm2ey': partial(calculate_mXey, missile_num=2),
    'm3eo': partial(calculate_mXeo, missile_num=3),
    'm3ey': partial(calculate_mXey, missile_num=3),
    'm1en': partial(calculate_elemental_damage_min, missile_num=1),
    'm1ex': partial(calculate_elemental_damage_max, missile_num=1),
    'm2en': partial(calculate_elemental_damage_min, missile_num=2),
    'm2ex': partial(calculate_elemental_damage_max, missile_num=2),
    'm3en': partial(calculate_elemental_damage_min, missile_num=3),
    'm3ex': partial(calculate_elemental_damage_max, missile_num=3),
    'mps': calculate_mana_cost_per_second,
    'usmc': partial(calculate_mana_cost, multiplier=256),
    'len': create_calculator_on_calc_field('auraLen'),
    'lvl': lambda skill, skill_levels, lvl: lvl,
    **{f'clc{i}': create_calculator_on_calc_field(f'calc{i}') for i in range(1, N_CALC_FIELDS + 1)},
    **{f'par{i}': partial(_get_param, param_number=i) for i in range(1, N_PARAMS + 1)},
}

## Define formatters

In [8]:
FRAMES_PER_SECOND = 25
YARDS_PER_GAME_UNIT = 2 / 3

def format_mana_cost(skill, skill_levels, lvl, ta, tb, ca, cb):
    cost = floor(calculate_mana_cost(skill, skill_levels, lvl), precision=ca or 10)
    return f"{skill['strMana']}{cost}" if cost else ''

def format_attack_rating(skill, skill_levels, lvl, ta, tb, ca, cb):
    attack_rating = calculate_to_hit(skill, skill_levels, lvl)
    if not attack_rating:
        return ''
    return f"Attack: +{attack_rating} percent"

def format_physical_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('minDam') is None:
        return ''

    min_damage = floor(calculate_physical_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_physical_damage_max(skill, skill_levels, lvl))

    if min_damage == max_damage:
        return f'Damage: +{min_damage}'
    
    return f'Damage: {min_damage}-{max_damage}'

def format_elemental_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    min_damage = floor(calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_elemental_damage_max(skill, skill_levels, lvl))

    if min_damage == max_damage:
        return f"{skill['eType']} Damage: +{min_damage}"
    
    return f"{skill['eType']} Damage: {min_damage}-{max_damage}"

def format_physical_damage_with_text(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('minDam') is None:
        return ''
    
    min_damage = floor(calculate_physical_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_physical_damage_max(skill, skill_levels, lvl))
    
    return f"{tb or ''}{ta or ''} {min_damage}-{max_damage}"

def format_elemental_damage_with_text(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    min_damage = floor(calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_elemental_damage_max(skill, skill_levels, lvl))
    
    return f"{tb or ''}{ta or ''} {min_damage}-{max_damage}"

def format_elemental_length(skill, skill_levels, lvl, ta, tb, ca, cb):
    if not skill.get('eLen'):
        return ''
    
    length = calculate_length(skill, skill_levels, lvl)
    formatted_length = int_if_no_decimals(length / FRAMES_PER_SECOND)
    return f"{skill['eType']} Length: {formatted_length} seconds"

def format_poison_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    adjustment = 1 if skill.get('hitShift') else 2 ** -8  # skills without hitshift 
    
    length = calculate_length(skill, skill_levels, lvl)
    min_damage = floor(length * calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(length * calculate_elemental_damage_max(skill, skill_levels, lvl))
    formatted_length = int_if_no_decimals(length / FRAMES_PER_SECOND)
    return f'Poison Damage: {min_damage}-{max_damage}\nover {formatted_length} seconds'

def format_elemental_damage_over_time(skill, skill_levels, lvl, ta, tb, ca, cb):
    min_damage = floor(FRAMES_PER_SECOND * calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(FRAMES_PER_SECOND * calculate_elemental_damage_max(skill, skill_levels, lvl))
    return f"Average {skill['eType']} Damage: {min_damage}-{max_damage} per second"

def format_elemental_damage_over_time_with_text(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = floor(calculate_skill_value(ca, skill, skill_levels, lvl))
    calcb = floor(calculate_skill_value(cb, skill, skill_levels, lvl))
    return f"{tb or ''}{ta or ''}{calca}-{calcb} per second"

def format_fire_missile_damage_over_time(skill, skill_levels, lvl, ta, tb, ca, cb):
    adjustment = 2.34 * 2 ** 5  # tuned, just higher than 7/3
    min_damage = floor(adjustment * calculate_elemental_damage_min(skill, skill_levels, lvl, missile_num=1))
    max_damage = floor(adjustment * calculate_elemental_damage_max(skill, skill_levels, lvl, missile_num=1))
    return f'Average Fire Damage: {min_damage}-{max_damage} per second'

def format_fire_damage_over_time(skill, skill_levels, lvl, ta, tb, ca, cb):
    adjustment = 2.34 * 2 ** 5  # tuned, just higher than 7/3
    min_damage = floor(adjustment * calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(adjustment * calculate_elemental_damage_max(skill, skill_levels, lvl))
    return f'Average Fire Damage: {min_damage}-{max_damage} per second'

def format_missile_range_in_yards(skill, skill_levels, lvl, ta, tb, ca, cb):
    missile_range = skill['missile1']['range'] + skill['missile1'].get('levRange', 0) * (lvl - 1)
    formatted_range = floor(missile_range * YARDS_PER_GAME_UNIT, 10)
    return f'{formatted_range} yards'

def format_missile_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    missile = skill['missile1']
    min_damage = floor(calculate_elemental_damage_min(missile, skill_levels, lvl))
    max_damage = floor(calculate_elemental_damage_max(missile, skill_levels, lvl))
    return f'{ta}{min_damage}-{max_damage}'

def format_missile_duration(skill, skill_levels, lvl, ta, tb, ca, cb):
    duration = skill['missile1']['range'] + skill['missile1'].get('levRange', 0) * (lvl - 1)
    formatted_duration = int_if_no_decimals(duration / FRAMES_PER_SECOND)
    return f'{ta}{formatted_duration} seconds'

def format_cltsubmissile_duration(skill, skill_levels, lvl, ta, tb, ca, cb):
    missile = skill['missile1']['cltSubMissile1']
    duration = missile['range'] + missile.get('levRange', 0) * (lvl - 1)
    formatted_duration = int_if_no_decimals(duration / FRAMES_PER_SECOND)
    return f'{ta}{formatted_duration} seconds'

def format_minion_life(skill, skill_levels, lvl, ta, tb, ca, cb):
    base_life = (skill['summon']['minHPNormal'] + skill['summon']['maxHPNormal']) / 2
    calca = calculate_skill_value(ca, skill, skill_levels, lvl) or 0
    calcb = calculate_skill_value(cb, skill, skill_levels, lvl) or 0
    
    life = floor((1 + calca / 100) * (base_life + calcb))
    return f'Life: {life}'

def format_skeleton_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    mastery_damage = skill_levels.get('skeletonMastery', 0) * skill['relatedSkills']['skeletonMastery']['params']['par2']
    monster_min_damage = skill['summon']['a1MinDNormal']
    monster_max_damage = skill['summon']['a1MaxDNormal']
    
    skill_damage = calculate_elemental_damage_min(skill, skill_levels, lvl)
    bonus = (lvl - 3) * skill['params']['par3'] if lvl > 3 else 0
    multiplier = 1 + bonus / 100

    min_damage = floor(multiplier * (monster_min_damage + skill_damage + mastery_damage))
    max_damage = floor(multiplier * (monster_max_damage + skill_damage + mastery_damage))

    return f'Damage: {min_damage}-{max_damage}'

def format_golem_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    damage_bonus = 0 if skill['strName'] == 'Iron Golem' else 35 * (lvl - 1)
    damage_multiplier = 1 + damage_bonus / 100
    
    monster_min_damage = skill['summon']['a1MinDNormal']
    monster_max_damage = skill['summon']['a1MaxDNormal']
    
    min_damage = floor(damage_multiplier * monster_min_damage)
    max_damage = floor(damage_multiplier * monster_max_damage)
    return f'Damage: {min_damage}-{max_damage}'

def format_fire_golem_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    monster_min_damage = skill['summon']['a1MinDNormal']
    monster_max_damage = skill['summon']['a1MaxDNormal']
    
    holy_fire_min_damage = calculate_skill_value(ca, skill, skill_levels, lvl)
    holy_fire_max_damage = calculate_skill_value(cb, skill, skill_levels, lvl)
    
    min_damage = floor(monster_min_damage + holy_fire_min_damage)
    max_damage = floor(monster_max_damage + holy_fire_max_damage)
    return f'Fire Damage: {min_damage}-{max_damage}'

def format_half_radius(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = calculate_skill_value(ca, skill, skill_levels, lvl)
    radius = floor(floor(calca) * YARDS_PER_GAME_UNIT / 2, precision=10)
    return f'{ta}{radius} yards'

def _fill_ta_with_tb(skill, skill_levels, lvl, ta, tb, ca, cb, pattern):
    return ta.replace(pattern, tb)

def _fill_ta_with_calca(skill, skill_levels, lvl, ta, tb, ca, cb, pattern):
    return ta.replace(pattern, str(calculate_skill_value(ca, skill, skill_levels, lvl)))

def format_text_on_calc_condition(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = floor(calculate_skill_value(ca, skill, skill_levels, lvl))
    return f'{calca}{ta if calca == 1 else tb}'

def create_simple_formatter(template):
    def formatter(skill, skill_levels, lvl, ta, tb, ca, cb):
        return template.format(ta=ta or '', tb=tb or '')
    return formatter

def create_calc_formatter(template, frames=False, game_units=False, precision=None, multiplier=None):
    def formatter(skill, skill_levels, lvl, ta, tb, ca, cb):
        calca = calculate_skill_value(ca, skill, skill_levels, lvl)
        calcb = calculate_skill_value(cb, skill, skill_levels, lvl)
        
        calca = _convert_calc(calca, frames, game_units, precision, multiplier)
        calcb = _convert_calc(calcb, frames, game_units, precision, multiplier)

        formatted = template.format(ta=ta or '', tb=tb or '', calca=calca, calcb=calcb)
        return formatted if calca or calcb else ''
    return formatter

def _convert_calc(calc, frames, game_units, precision, multiplier):
    if calc is None:
        return calc
    if frames:
        calc /= FRAMES_PER_SECOND
    if game_units:
        calc = floor(calc) * YARDS_PER_GAME_UNIT
    if multiplier is not None:
        calc *= multiplier
    if precision is not None:
        calc = floor(calc, precision)
    return calc

In [9]:
FORMATTERS_BY_DESCLINE = {
    1: format_mana_cost,
    2: create_calc_formatter('{ta} {calca:+d}{tb}', precision=1),
    3: create_calc_formatter('{ta}{calca}{tb}', precision=1),
    4: create_calc_formatter('{ta}+{calca}'),
    5: create_calc_formatter('{ta} {calca}', precision=1),
    6: create_calc_formatter('+{calca} {ta}'),
    7: create_calc_formatter('{calca} {ta}'),
    8: format_attack_rating,
    9: format_physical_damage,
    10: format_elemental_damage,
    11: format_elemental_length,
    12: create_calc_formatter('{ta}{calca} seconds', frames=True, precision=10),
    13: format_minion_life,
    14: format_poison_damage,
    15: create_simple_formatter('{ta}:{tb}'),  #not used
    16: create_calc_formatter('Duration: {calca}-{calcb} seconds', frames=True),
    17: format_elemental_damage_over_time_with_text,
    18: create_simple_formatter('{ta}'),
    19: create_calc_formatter('{tb}{ta} {calca} yards', game_units=True, precision=10),
    20: create_calc_formatter('{ta}+{calca} percent {tb}'),  # not used
    21: create_calc_formatter('{ta}{calca} percent {tb}'),  # not used
    22: format_fire_missile_damage_over_time,
    23: format_missile_duration,
    24: format_elemental_damage_with_text,
    25: create_simple_formatter('{ta}{tb}'),
    26: format_elemental_damage_over_time,
    27: format_fire_damage_over_time,
    28: create_simple_formatter('Radius: 1 yard'),
    29: format_missile_range_in_yards,
    30: format_cltsubmissile_duration,
    31: create_calc_formatter('{ta} {calca} seconds', frames=True, precision=10),
    32: create_calc_formatter('{ta}{tb}+{calca} percent'),
    33: create_simple_formatter('{ta}{tb}'),
    34: format_skeleton_damage,
    35: create_calc_formatter('{ta}: {calca}-{calcb}', precision=1),
    36: format_text_on_calc_condition,
    37: format_half_radius,
    38: create_calc_formatter('{ta}{calca}-{calcb}{tb}', precision=1),
    39: format_golem_damage,
    40: partial(_fill_ta_with_tb, pattern='%s'),  # ca refers to color, ignored here
    41: format_fire_golem_damage,
    42: create_calc_formatter('{ta}: +{calca}.{calcb} {tb}', precision=1),
    43: create_calc_formatter('{ta}{calca}-{calcb}{tb}', precision=10, multiplier=1/256),
    44: create_calc_formatter('{ta}{calca}-{calcb}{tb}', precision=10, multiplier=1/256),  # not used
    # 45: not used
    46: create_simple_formatter('{ta}'),  # not used
    47: create_calc_formatter('{ta}{calca}-{calcb}', precision=1),
    48: format_elemental_damage,
    49: format_physical_damage_with_text,
    50: format_missile_damage,
    51: partial(_fill_ta_with_calca, pattern='%d'),
    52: create_calc_formatter('{ta}+{calca}-{calcb}{tb}', precision=1),
    #53: not used
    #54: not used
    #55: not used
    #56: not used
    57: create_calc_formatter('{ta}+{calca} seconds', frames=True, precision=1),
    58: create_calc_formatter('{ta}{tb}+{calca}-{calcb}', precision=1),  # not used
    59: create_calc_formatter('{tb}{ta}{calca}-{calcb}', precision=1),
    60: create_calc_formatter('{ta}+{calcb}{tb}'),  # not used, 256 precision
    61: create_calc_formatter('{ta}{calca}{tb}', precision=10, multiplier=1/256),
    62: create_calc_formatter('{ta}{tb}{calca}-{calcb}', precision=1),
    63: create_calc_formatter('{ta}: +{calca}% {tb}'),
    64: create_calc_formatter('{ta}: +{calca}/{calcb} {tb}', precision=1),  #not used
    65: create_simple_formatter('{ta}: {tb}'),  # not used
    66: partial(_fill_ta_with_calca, pattern='%d%'),
    67: create_calc_formatter('{ta}: +{calca} {tb}', precision=1),
    68: create_calc_formatter('{calca}{ta}{tb}'),
    69: create_calc_formatter('{ta}: {tb} {calca}'),  # not used
    70: create_calc_formatter('{ta}{tb}+{calca}'),
    71: create_simple_formatter('{ta}: {tb}'),
    72: create_calc_formatter('+{calca}/{calcb} {ta}'),
    73: create_calc_formatter('{calca}/{calcb} {ta}'),
    74: create_simple_formatter('{ta}'),  # not used
    75: create_simple_formatter('{ta}'),  #not used
}

def format_descline(descline_number, skill, skill_levels, lvl, ta, tb, ca, cb):
    formatter = FORMATTERS_BY_DESCLINE[descline_number]
    return formatter(skill, skill_levels, lvl, ta, tb, ca, cb)

## Format skills

In [10]:
def format_skill(skill, skill_levels, lvl):
    lines = [
        skill['strName'],
        skill['strLong'],
        '',
        format_desclines('dsc2', skill, skill_levels, lvl),
        f'Current Level: {lvl}',
        format_desclines('desc', skill, skill_levels, lvl),
        f'Next Level: {lvl + 1}',
        format_desclines('desc', skill, skill_levels, lvl + 1),
        format_desclines('dsc3', skill, skill_levels, lvl),
    ]
    return '\n'.join([line for line in lines if line is not None])

def format_desclines(root, skill, skill_levels, lvl):
    lines = []
    for entry in skill.get(f'{root}Lines', []):
        line = format_descline(
            descline_number=entry[f'{root}Line'],
            skill=skill,
            skill_levels=skill_levels,
            lvl=lvl,
            ta=entry.get(f'{root}TextA'),
            tb=entry.get(f'{root}TextB'),
            ca=entry.get(f'{root}CalcA'),
            cb=entry.get(f'{root}CalcB'),
        )
        if line:
            lines.append(line)
    return '\n'.join(lines) + '\n' if lines else None

In [11]:
# print(format_skill(skill_details['summonSpiritWolf'], {}, 10))

In [12]:
# skill_details['summonSpiritWolf']

# Format all skills

In [13]:
for skill_name, skill in skill_details.items():
    print(format_skill(skill, {'clayGolem': 1}, 10))
    print('------------------------------------')
    print('------------------------------------')
    print()

Magic Arrow
creates a magical arrow or bolt
that does extra damage

Current Level: 10
Converts 10% Physical Damage to Magic Damage
Attack: +91 percent
Damage: +10
Mana Cost: 0.3

Next Level: 11
Converts 11% Physical Damage to Magic Damage
Attack: +100 percent
Damage: +11
Mana Cost: 0.2

------------------------------------
------------------------------------

Fire Arrow
magically enhances your arrows
or bolts with fire

Current Level: 10
Converts 21% Physical Damage to Elemental Damage
Attack: +91 percent
Fire Damage: 21-24
Mana Cost: 4.1

Next Level: 11
Converts 23% Physical Damage to Elemental Damage
Attack: +100 percent
Fire Damage: 24-27
Mana Cost: 4.2

Fire Arrow Receives Bonuses From:
Exploding Arrow: +12% Fire Damage per Level

------------------------------------
------------------------------------

Inner Sight
illuminates nearby enemies
making them easier to hit
for you and your party

Current Level: 10
Duration: 44 seconds
Enemy Defense:  -305
Radius:  13.3 yards
Mana Cost:

## Look up skills by descline for development

In [14]:
def select_skills_with_descline(skill_details, pattern):
    skill_names = []
    for skill_name, skill in skill_details.items():
        for descline in skill['descLines']:
            if descline['descLine'] == pattern:
                skill_names.append(skill_name)
                continue
    return skill_names

select_skills_with_descline(skill_details, 31)

['dimVision', 'confuse', 'attract']